| Feature              | Description                                               |
| -------------------- | --------------------------------------------------------- |
| **Traffic Movement** | Animated flow of traffic density on NYC roads             |
| **Weather Changes**  | Real-time weather overlay (clouds, rain, temp) using APIs |
| **Pedestrian Flow**  | Simulated or real pedestrian count on hotspots            |
| **City Soundscapes** | Area-wise decibel levels mapped to heatmaps               |
| **Pollution Levels** | AQI zone-wise heatmap (CO2, PM2.5)                        |
| **Urban Heat Zones** | Based on temperature & density mapped spatially           |


In [1]:
# Install required packages
!pip install folium pydeck geopandas requests openmeteo-requests plotly

# Imports
import folium
import pydeck as pdk
import pandas as pd
import numpy as np
import requests
import plotly.express as px
from folium.plugins import HeatMap


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 669.8/669.8 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.9/247.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.4/393.4 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.2 MB/s eta 0:00:00


# **Dummy Data**

In [31]:
nyc_lat, nyc_lon = 40.7128, -74.0060

def generate_points(n, lat, lon, radius=0.03):
    return [(lat + np.random.uniform(-radius, radius), lon + np.random.uniform(-radius, radius)) for _ in range(n)]

traffic_data = pd.DataFrame(generate_points(300, nyc_lat, nyc_lon), columns=['lat', 'lon'])
traffic_data['intensity'] = np.random.randint(10, 100, size=300)


# **Traffic Visualization**

In [3]:
traffic_map = folium.Map(location=[nyc_lat, nyc_lon], zoom_start=13)
HeatMap(data=traffic_data[['lat', 'lon', 'intensity']].values.tolist(), radius=10).add_to(traffic_map)
traffic_map.save("traffic_map.html")
traffic_map



# **Simulated Pedestrian Flow on Mapbox with pydeck**

In [19]:
pedestrian_data = pd.DataFrame(generate_points(200, nyc_lat, nyc_lon), columns=['lat', 'lon'])
pedestrian_data['count'] = np.random.randint(1, 50, size=200)

layer = pdk.Layer(
    "ScatterplotLayer",
    data=pedestrian_data,
    get_position='[lon, lat]',
    get_color='[255, 140, 0, 160]',
    get_radius='count * 3',
)

view_state = pdk.ViewState(latitude=nyc_lat, longitude=nyc_lon, zoom=12)

r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.show()



<IPython.core.display.Javascript object>

# **City Soundscape**

In [30]:
sound_data = pd.DataFrame(generate_points(100, nyc_lat, nyc_lon), columns=['lat', 'lon'])
sound_data['decibel'] = np.random.randint(40, 100, size=100)

fig = px.density_mapbox(sound_data, lat='lat', lon='lon', z='decibel',
                        radius=20, center=dict(lat=nyc_lat, lon=nyc_lon),
                        zoom=12, mapbox_style="stamen-terrain", color_continuous_scale='Inferno')

fig.show()


# **Pollution Zones**

In [21]:
pollution_data = pd.DataFrame(generate_points(150, nyc_lat, nyc_lon), columns=['lat', 'lon'])
pollution_data['AQI'] = np.random.randint(50, 300, size=150)

fig = px.density_mapbox(pollution_data, lat='lat', lon='lon', z='AQI',
                        radius=25, center=dict(lat=nyc_lat, lon=nyc_lon),
                        zoom=12, mapbox_style="carto-positron", color_continuous_scale='YlOrRd')

fig.show()


# **Urban Heat Zones**

In [22]:
heat_data = pd.DataFrame(generate_points(100, nyc_lat, nyc_lon), columns=['lat', 'lon'])
heat_data['temp'] = np.random.uniform(28, 45, size=100)

fig = px.scatter_mapbox(heat_data, lat='lat', lon='lon', color='temp',
                        color_continuous_scale='Oranges',
                        size_max=15, zoom=11, mapbox_style='open-street-map')

fig.show()
